# Introduction
In the below analysis, I'm trying to the Calculate Customer return Value using different predicting models.

# BD-NBD model:
*Beta Geometric / Negative Binomial Distribution known as BG-NBD Model. Also sometimes it comes up as “Buy Till You Die”.*
1.  Transaction Process (Buy) -Gamma Distribution
2.  Dropout Process (Till You Die)- Beta Distribution

# Gamma-Gamma Submodel:
*We use this model for predicting how much average profit we can earn for each customer. It gives us the expected average profit for each customer after modelling the average profit for the mass.*



In [ ]:
!pip install lifetimes
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.7 MB/s eta 0:00:00


In [31]:
#Include the required package
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

# Loading Data

In [32]:
raw_data = pd.read_excel('/online_retail_II.xlsx',sheet_name='Year 2010-2011')

In [ ]:
df = raw_data.copy()

In [ ]:
#Get a glimpse of the data
raw_data.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
# Use .info() for a concise summary of the DataFrame
df.info()

# Use .describe() for descriptive statistics of numerical columns
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,Quantity,InvoiceDate,Price,Customer ID
count,541910.000000,541910,541910.000000,406830.000000
mean,9.552234,2011-07-04 13:35:22.342307584,4.611138,15287.684160
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.080957,NaN,96.759765,1713.603074


In [ ]:
#Get the list of columns with the data types
print(df.dtypes)

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object


In [ ]:
# Get the shape of the DataFrame (rows, columns)
print(f"Shape of the dataset:{df.shape}")

# View unique values in a specific column
print(f"No_of_unique_customer_ids:{len(df['Customer ID'].unique())}")



Shape of the dataset:(541910, 8)
No_of_unique_customer_ids:4373


In [ ]:
# Get a summary of missing values
print(df.isnull().sum())

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


In [ ]:
#Drop the null values
df.dropna(inplace=True)
print(df.isnull().sum())

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64


In [ ]:
# Filter the rows with a true Quantity Value
df=df[df["Quantity"] > 0]

In [ ]:
df["Invoice"] = df["Invoice"].astype(str)  # Convert to string
df = df[~df["Invoice"].str.contains("C", na=False)]


# Data Preparing

In [ ]:
# Create a new column as Total Price
df['TotalPrice'] = df['Price'] * df['Quantity']


**Recency (R):**  This measures how recently a customer made a purchase.

The more recent the purchase, the higher the recency score.

**Formula:**
*Recency=Current Date-Date of Last Purchase(Max date-Min date)*

**Frequency (F):** This measures how often a customer makes a purchase during a specified time period. Higher frequency indicates a more engaged customer.

**Formula:**
*Frequency=Total Number of Purchases(unique Invoices)*

**Monetary (M):** This measures how much money a customer spends during a specified time period. Higher monetary values indicate more valuable customers.

**Formula:**
*Monetary =Total Revenue Generated by the Customer(Total value)*


In [35]:
# # Convert 'InvoiceDate' to datetime
# df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# # Define the current date
# current_date = df['InvoiceDate'].max()

# # Calculate Recency
# recency_df = df.groupby('Customer ID').agg({'InvoiceDate': 'max'}).reset_index()
# recency_df['Recency'] = (current_date - recency_df['InvoiceDate']).dt.days

# # Calculate Frequency
# frequency_df = df.groupby('Customer ID').size().reset_index(name='Frequency')

# # Calculate Monetary
# monetary_df = df.groupby('Customer ID')['TotalPrice'].sum().reset_index()

# # Merge all metrics into one DataFrame
# rfm_df = recency_df.merge(frequency_df, on='Customer ID').merge(monetary_df, on='Customer ID')


# Preparing Dataset for Calculating CLTV

In [42]:
# Random to find the timeline
today_date = dt.datetime(2011, 12, 11)

cltv = df.groupby('Customer ID').agg({
    'InvoiceDate': [
        lambda x: (x.max() - x.min()).days,  # recency
        lambda x: (today_date - x.min()).days  #  Timeline indicates how long the customer has been active with the company
    ],
    'Invoice': lambda x: x.nunique(),  # frequency
    'TotalPrice': lambda x: x.sum()  # monetary
})

cltv.columns = cltv.columns.droplevel(0)

cltv.columns = ['recency', 'Timeline', 'frequency', 'monetary']

cltv = cltv[cltv['monetary'] > 0]

# in this case we will suppose that monetary value is average earning per transaction
cltv['monetary'] = cltv['monetary'] / cltv['frequency']


# transforming days to weeks
cltv['recency'] = cltv['recency'] / 7
cltv['Timeline'] = cltv['Timeline'] / 7

cltv = cltv[(cltv['frequency'] > 1)]


# Creating BG-NBD Model

In [44]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv['frequency'], cltv['recency'], cltv['Timeline'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.40, b: 2.49, r: 2.18>

## Asking Some Questions and Answering Them

In [46]:
# Top 10 customers expected to make the most purchases in a week
bgf.conditional_expected_number_of_purchases_up_to_time(1, # week
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['Timeline']).sort_values(ascending=False).head(10)

,0
Customer ID,
12748.0,3.265158
14911.0,3.126645
17841.0,1.940290
13089.0,1.537528
14606.0,1.463999
15311.0,1.433717
12971.0,1.357024
14646.0,1.222517
13408.0,0.986249


In [48]:
# Top 10 customers expected to make the most purchases in a month
bgf.conditional_expected_number_of_purchases_up_to_time(4, # 4 weeks = 1 month
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['Timeline']).sort_values(ascending=False).head(10)

,0
Customer ID,
12748.0,13.025670
14911.0,12.473095
17841.0,7.740345
13089.0,6.133456
14606.0,5.840269
15311.0,5.719467
12971.0,5.413481
14646.0,4.876360
13408.0,3.934373


In [50]:
# Top 10 customers expected to make the most purchases in next 6 months
bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6, # weeks * count = months
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['Timeline']).sort_values(ascending=False).head(10)

,0
Customer ID,
12748.0,76.909602
14911.0,73.646821
17841.0,45.701412
13089.0,36.208661
14606.0,34.482056
15311.0,33.768757
12971.0,31.961007
14646.0,28.772623
13408.0,23.228346


In [51]:
# The count of total transactions expected in next 6 months
bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6,
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['Timeline']).sum()

10394.315175532292

# Creating Gamma Gamma Model

In [52]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv['frequency'], cltv['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.79, q: 0.34, v: 3.72>

## Asking Questions and Aswering them

In [53]:
# The top 10 customers expected to be most valuable
ggf.conditional_expected_average_profit(cltv['frequency'],
                                        cltv['monetary']).sort_values(ascending=False).head(10)

,0
Customer ID,
16446.0,92300.885839
15749.0,15764.032180
15098.0,14129.582601
12415.0,5998.390804
12590.0,5406.242392
12536.0,4461.677780
16000.0,4388.011101
18102.0,4340.321694
12435.0,4291.698433


# Predicting CLTV

In [55]:
# The customers' lifetime values expected to in the next 3 months
cltv['cltv_pred_3_months'] = ggf.customer_lifetime_value(bgf,
                                   cltv['frequency'],
                                   cltv['recency'],
                                   cltv['Timeline'],
                                   cltv['monetary'],
                                   time=3,  # 3 months
                                   freq="W",  # frequency information of T. In this case we set week by using 'W'
                                   discount_rate=0.01)
cltv

,recency,Timeline,frequency,monetary,cltv_pred_3_months
Customer ID,,,,,
12347.0,52.142857,52.571429,7,615.714286,1128.653739
12348.0,40.285714,51.285714,4,449.310000,546.997685
12352.0,37.142857,42.428571,8,313.255000,738.058132
12356.0,43.142857,46.571429,3,937.143333,1083.507270
12358.0,21.285714,21.571429,2,584.030000,982.080404
...,...,...,...,...,...
18272.0,34.857143,35.285714,6,513.096667,1146.470687
18273.0,36.428571,36.857143,3,68.000000,96.599662
18282.0,16.857143,18.142857,2,89.025000,168.698793


## Segmenting Customers by Using Predicted CLTV

In [56]:
cltv['segment'] = pd.qcut(cltv['cltv_pred_3_months'],4,['D','C','B','A']) # Segregate the customer into 4 Quantiles(Top 25%)

cltv

,recency,Timeline,frequency,monetary,cltv_pred_3_months,segment
Customer ID,,,,,,
12347.0,52.142857,52.571429,7,615.714286,1128.653739,A
12348.0,40.285714,51.285714,4,449.310000,546.997685,B
12352.0,37.142857,42.428571,8,313.255000,738.058132,B
12356.0,43.142857,46.571429,3,937.143333,1083.507270,A
12358.0,21.285714,21.571429,2,584.030000,982.080404,A
...,...,...,...,...,...,...
18272.0,34.857143,35.285714,6,513.096667,1146.470687,A
18273.0,36.428571,36.857143,3,68.000000,96.599662,D
18282.0,16.857143,18.142857,2,89.025000,168.698793,D


In [69]:
##These are top 5 customers with higher purchase average in th following 3 months
cltv[cltv['segment']=='A']['cltv_pred_3_months'].sort_values(ascending=False).head(5)

,cltv_pred_3_months
Customer ID,
16446.0,114514.629477
14646.0,58663.166838
18102.0,53164.411473
17450.0,40130.337698
14096.0,35220.462461


Conclusion:

 By doing the Customer Lifetime Value analysis we can understand the customer behavior and to adopt bettwe marketing strategies.

** Use-cases**
*Marketing Strategy:* By understanding which customers are more valuable, businesses can tailor their marketing efforts to focus on high-value segments, optimizing their return on investment.


*Resource Allocation:* Resources can be allocated more efficiently by prioritizing high-value customers for engagement and retention efforts.
(If we have a product table details in the following data set we can do a product level analysis for it).

Done and Dusted!!!
Source : https://medium.com/analytics-vidhya/customer-life-time-value-prediction-by-using-bg-nbd-gamma-gamma-models-and-applied-example-in-997a5ee481ad
